In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import scipy.stats
import math
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelBinarizer


file_names = [
    '/content/0.7inner-100watt.csv', '/content/0.7outer-100watt.csv',
    '/content/0.7inner-200watt.csv', '/content/0.7outer-200watt.csv',
    '/content/0.7inner-300watt.csv', '/content/0.7outer-300watt.csv',
    '/content/0.9inner-100watt.csv', '/content/0.9outer-100watt.csv',
    '/content/0.9inner-200watt.csv', '/content/0.9outer-200watt.csv',
    '/content/0.9inner-300watt.csv', '/content/0.9outer-300watt.csv',
    '/content/1.1inner-100watt.csv', '/content/1.1outer-100watt.csv',
    '/content/1.1inner-200watt.csv', '/content/1.1outer-200watt.csv',
    '/content/1.1inner-300watt.csv', '/content/1.1outer-300watt.csv',
    '/content/1.3inner-100watt.csv', '/content/1.3outer-100watt.csv',
    '/content/1.3inner-200watt.csv', '/content/1.3outer-200watt.csv',
    '/content/1.3inner-300watt.csv', '/content/1.3outer-300watt.csv',
    '/content/1.5inner-100watt.csv', '/content/1.5outer-100watt.csv',
    '/content/1.5inner-200watt.csv', '/content/1.5outer-200watt.csv',
    '/content/1.5inner-300watt.csv', '/content/1.5outer-300watt.csv',
    '/content/1.7inner-100watt.csv', '/content/1.7outer-100watt.csv',
    '/content/1.7inner-200watt.csv', '/content/1.7outer-200watt.csv',
    '/content/1.7inner-300watt.csv', '/content/1.7outer-300watt.csv',
    '/content/BRB-12-4-100watt.csv', '/content/BRB-12-4-300watt.csv'
    ,'/content/healthy.csv'
]


combined_reshaped_data = pd.DataFrame()

# Iterate over the list of file names
for file_name in file_names:
    # Read the CSV file
    data = pd.read_csv(file_name)
    label = 'healthy' if 'healthy.csv' in file_name else 'unhealthy'  # Label based on file name


    # Select the "Current-A" column
    current_a_column = data[' Current-A']

    # Transpose the "Current-A" column
    transposed_current_a = current_a_column.head(100000).values.reshape(1, -1)

    # Reshape the transposed "Current-A" column into blocks of 1000 columns for each row
    block_size = 1000
    num_blocks = transposed_current_a.shape[1] // block_size
    reshaped_current_a = transposed_current_a[:, :num_blocks * block_size].reshape(-1, block_size)

    # Create a DataFrame from the reshaped "Current-A" column
    reshaped_current_a_df = pd.DataFrame(reshaped_current_a, columns=[f'Block_{i+1}' for i in range(block_size)])

    # Add a new column 'Label' with the corresponding label for each row
    reshaped_current_a_df['Label'] = label

    # Append the reshaped data to the combined DataFrame
    combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)

# Save the combined reshaped data to a new CSV file
a = combined_reshaped_data.to_csv('/content/final_combined.csv', index=False)

print(combined_reshaped_data)
combined_reshaped_data.shape

<ipython-input-18-d22d1ba80f72>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)
<ipython-input-18-d22d1ba80f72>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)
<ipython-input-18-d22d1ba80f72>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)
<ipython-input-18-d22d1ba80f72>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = 

      Block_1  Block_2  Block_3  Block_4  Block_5  Block_6  Block_7  Block_8  \
0      2.3309   2.3309   2.3309   2.3309   2.3309   2.3309   2.3309   2.3309   
1      2.7228   2.7228   2.7228   2.7228   2.7228   2.7228   2.7228   2.7228   
2      2.9524   2.9524   2.9524   2.9524   2.9524   2.9524   2.9524   2.9524   
3      2.1111   2.1111   2.1111   2.1111   2.1111   2.1111   2.1111   2.1111   
4      2.1282   2.1282   2.1282   2.1282   2.1282   2.1282   2.1282   2.1282   
...       ...      ...      ...      ...      ...      ...      ...      ...   
3861   2.4603   2.4530   2.4530   2.4530   2.4530   2.4530   2.4530   2.4530   
3862   2.3712   2.3712   2.3712   2.3712   2.3712   2.3712   2.3712   2.3712   
3863   2.3065   2.3065   2.3065   2.3065   2.3065   2.3065   2.3065   2.3065   
3864   2.2552   2.2552   2.2552   2.2552   2.2552   2.2552   2.2552   2.2552   
3865   2.2479   2.2479   2.2479   2.2479   2.2479   2.2479   2.2479   2.2479   

      Block_9  Block_10  ...  Block_992

(3866, 1001)

In [4]:
final = pd.read_csv('/content/final_combined.csv')
Fdf = pd.DataFrame(final)
Fdf.shape

(3866, 1001)

In [5]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

class LogisticRegression():
    def __init__(self, lr=0.001, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            linear_pred = np.dot(X, self.weights) + self.bias
            A_pred = sigmoid(linear_pred)

            dw = (1 / n_samples) * np.dot(X.T, (A_pred - y))
            db = (1 / n_samples) * np.sum(A_pred - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_pred = np.dot(X, self.weights) + self.bias
        A_pred = sigmoid(linear_pred)
        class_pred = [1 if p >= 0.5 else 0 for p in A_pred]
        return class_pred

In [6]:
X = final.iloc[:, :-1].values  # Assuming the last column is the target variable
y = final.iloc[:, -1].values
# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression(lr=0.01)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9728682170542635


In [15]:
# List of file names
file_names = [
    '/content/0.7inner-100watt.csv', '/content/0.7outer-100watt.csv',
    '/content/0.7inner-200watt.csv', '/content/0.7outer-200watt.csv',
    '/content/0.7inner-300watt.csv', '/content/0.7outer-300watt.csv',
    '/content/0.9inner-100watt.csv', '/content/0.9outer-100watt.csv',
    '/content/0.9inner-200watt.csv', '/content/0.9outer-200watt.csv',
    '/content/0.9inner-300watt.csv', '/content/0.9outer-300watt.csv',
    '/content/1.1inner-100watt.csv', '/content/1.1outer-100watt.csv',
    '/content/1.1inner-200watt.csv', '/content/1.1outer-200watt.csv',
    '/content/1.1inner-300watt.csv', '/content/1.1outer-300watt.csv',
    '/content/1.3inner-100watt.csv', '/content/1.3outer-100watt.csv',
    '/content/1.3inner-200watt.csv', '/content/1.3outer-200watt.csv',
    '/content/1.3inner-300watt.csv', '/content/1.3outer-300watt.csv',
    '/content/1.5inner-100watt.csv', '/content/1.5outer-100watt.csv',
    '/content/1.5inner-200watt.csv', '/content/1.5outer-200watt.csv',
    '/content/1.5inner-300watt.csv', '/content/1.5outer-300watt.csv',
    '/content/1.7inner-100watt.csv', '/content/1.7outer-100watt.csv',
    '/content/1.7inner-200watt.csv', '/content/1.7outer-200watt.csv',
    '/content/1.7inner-300watt.csv', '/content/1.7outer-300watt.csv',
    '/content/BRB-12-4-100watt.csv', '/content/BRB-12-4-300watt.csv'
    ,'/content/healthy.csv'
]

# Initialize an empty DataFrame to store the reshaped data
combined_reshaped_data = pd.DataFrame()

# Iterate over the list of file names
for file_name in file_names:
    # Read the CSV file
    data = pd.read_csv(file_name)
    label = file_name.split('/')[-1].split('-')[0]  # Extract the label from the file name

    # Select the "Current-A" column
    current_a_column = data[' Current-A']

    # Transpose the "Current-A" column
    transposed_current_a = current_a_column.head(100000).values.reshape(1, -1)

    # Reshape the transposed "Current-A" column into blocks of 1000 columns for each row
    block_size = 1000
    num_blocks = transposed_current_a.shape[1] // block_size
    reshaped_current_a = transposed_current_a[:, :num_blocks * block_size].reshape(-1, block_size)

    # Create a DataFrame from the reshaped "Current-A" column
    reshaped_current_a_df = pd.DataFrame(reshaped_current_a, columns=[f'Block_{i+1}' for i in range(block_size)])

    # Add a new column 'Label' with the corresponding label for each row
    reshaped_current_a_df['Label'] = label

    # Append the reshaped data to the combined DataFrame
    combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)

# Save the combined reshaped data to a new CSV file
a = combined_reshaped_data.to_csv('/content/final1_combined.csv', index=False)

# Display the combined reshaped DataFrame
print(combined_reshaped_data)
combined_reshaped_data.shape

<ipython-input-15-0d552e982bd2>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)
<ipython-input-15-0d552e982bd2>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)
<ipython-input-15-0d552e982bd2>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = combined_reshaped_data.append(reshaped_current_a_df, ignore_index=True)
<ipython-input-15-0d552e982bd2>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_reshaped_data = 

      Block_1  Block_2  Block_3  Block_4  Block_5  Block_6  Block_7  Block_8  \
0      2.3309   2.3309   2.3309   2.3309   2.3309   2.3309   2.3309   2.3309   
1      2.7228   2.7228   2.7228   2.7228   2.7228   2.7228   2.7228   2.7228   
2      2.9524   2.9524   2.9524   2.9524   2.9524   2.9524   2.9524   2.9524   
3      2.1111   2.1111   2.1111   2.1111   2.1111   2.1111   2.1111   2.1111   
4      2.1282   2.1282   2.1282   2.1282   2.1282   2.1282   2.1282   2.1282   
...       ...      ...      ...      ...      ...      ...      ...      ...   
3861   2.4603   2.4530   2.4530   2.4530   2.4530   2.4530   2.4530   2.4530   
3862   2.3712   2.3712   2.3712   2.3712   2.3712   2.3712   2.3712   2.3712   
3863   2.3065   2.3065   2.3065   2.3065   2.3065   2.3065   2.3065   2.3065   
3864   2.2552   2.2552   2.2552   2.2552   2.2552   2.2552   2.2552   2.2552   
3865   2.2479   2.2479   2.2479   2.2479   2.2479   2.2479   2.2479   2.2479   

      Block_9  Block_10  ...  Block_992

(3866, 1001)

In [16]:
def softmax(z):
    if z.ndim == 1:
        # If z is a one-dimensional array, treat it as a single sample
        exp_scores = np.exp(z - np.max(z))
        probs = exp_scores / np.sum(exp_scores)
    else:
        # If z is a matrix, apply softmax to each row
        exp_scores = np.exp(z - np.max(z, axis=1, keepdims=True))
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return probs

class MultiClassLogisticRegression():
    def __init__(self, lr=0.001, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            linear_pred = np.dot(X, self.weights) + self.bias
            A_pred = softmax(linear_pred)

            dw = (1 / n_samples) * np.dot(X.T, (A_pred - y))
            db = (1 / n_samples) * np.sum(A_pred - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_pred = np.dot(X, self.weights) + self.bias
        A_pred = softmax(linear_pred)
        class_pred = [1 if p >= 0.5 else 0 for p in A_pred]
        return class_pred
        #class_pred = np.argmax(A_pred,axis=1)
        #return class_pred

In [21]:
final1 = pd.read_csv('/content/final1_combined.csv')
X = final1.iloc[:, :-1].values  # Assuming the last column is the target variable
y = final1.iloc[:, -1].values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = MultiClassLogisticRegression(lr=0.01)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.09819121447028424
